In [136]:
import re
import requests
from bs4 import BeautifulSoup
import sqlite3
from pymystem3 import Mystem
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from thisapidoesnotexist import get_cat
from matplotlib import pyplot as plt

conn = sqlite3.connect('stihi.db')
c = conn.cursor()
m = Mystem()
c.execute("CREATE TABLE IF NOT EXISTS 'Stihi'('id' integer PRIMARY KEY AUTOINCREMENT, 'word1', 'lemma1', 'pos1', 'word2', 'lemma2', 'pos2', 'word3', 'lemma3', 'pos3', 'texts' text, 'url')")

Функция, достающая стихи про животных с сайта Rustih.ru.

In [2]:
def get_texts():
    texts = []
    hrefs = []
    response = requests.get('https://rustih.ru/stixi-o-zhivotnyx/', headers={'User-agent':'Mozilla/5.0'}).text
    soup = BeautifulSoup(response,'html.parser')
    et = soup.find_all('div', {'class': 'entry-title'})
    for i in et:
        hr = i.find('a').attrs['href']
        hrefs.append(hr)
        response = requests.get(hr, headers={'User-agent':'Mozilla/5.0'}).text
        soup = BeautifulSoup(response,'html.parser')
        pt = soup.find('div', {'class': 'entry-content poem-text'})
        text = pt.find_all(['p', 'h2'])
        t = ''
        for j in text:
            if 'adsbygoogle' not in j.text:
                if 'I' not in j.text:
                    clt = re.sub('\\u200e', '', j.text)
                    t += clt + ' '
            else:
                break
        texts.append(t)

    return texts, hrefs

In [3]:
texts, hrefs = get_texts()

Функция, заносящая тексты в базу в нужном формате.
Колонки в таблице: word1, lemma1, pos1, word2, lemma2, pos2, word3, lemma3, pos3, texts, url.

In [137]:
def make_db(texts, hrefs):
    for i in tqdm(range(len(texts))):
        sents = sent_tokenize(texts[i])
        for s in sents:
            ana = []
            for w in m.analyze(s):
                if 'analysis' in w:
                    if w['analysis'] == []:
                        pos = ''
                        lemma = ''
                    elif 'gr' not in w['analysis'][0]:
                        pos = ''
                    elif 'lex' not in w['analysis'][0]:
                        lemma = ''
                    else:
                        gr = w['analysis'][0]['gr']
                        pos = gr.split('=')[0].split(',')[0]
                        lemma = w['analysis'][0]['lex']
                    ana.append((w['text'].lower(), lemma, pos))
            ana.extend([('', '', ''), ('', '', '')])
                    
            for j in range(len(ana[:-2])):
                v = (ana[j][0], ana[j][1], ana[j][2], ana[j+1][0], ana[j+1][1], ana[j+1][2], ana[j+2][0], ana[j+2][1], ana[j+2][2], s, hrefs[i])
                c.execute("INSERT INTO Stihi (word1, lemma1, pos1, word2, lemma2, pos2, word3, lemma3, pos3, texts, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", v)
                conn.commit()


In [138]:
make_db(texts, hrefs)


100%|██████████| 115/115 [04:20<00:00,  2.27s/it]


In [153]:
conn.close()